### Build a Simple LLM Application with LCEL

LCEL in the context of LLMs (Large Language Models) typically refers to "Language Chain Execution Layer" or sometimes "Low-Level Chain Execution Layer", which is a component within the LangChain framework.


In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe


### What is LCEL?

LCEL stands for LangChain Expression Language. It is a powerful tool within LangChain that allows developers to chain components together in a structured and efficient manner. LCEL simplifies the process of building complex workflows by enabling seamless integration of language models, prompt templates, and output parsers.

With LCEL, you can:

- Combine multiple components into a single workflow.
- Debug and trace your application effectively using LangSmith.
- Deploy your application easily with LangServe.

LCEL is particularly useful for creating modular and reusable components in your LLM applications.


In [1]:
### Open AI API Key and Open Source Models --Llama2, Gemma, Mistral --Groq
import os
from dotenv import load_dotenv
load_dotenv()

import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

groq_api_key = os.getenv("GROQ_API_KEY")

In [2]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq

model = ChatGroq(model ="gemma2-9b-it", groq_api_key=groq_api_key)

In [3]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(
        content="Translate the following English text to French"),
    HumanMessage(
        content="Hello, how are you?")
]

response = model.invoke(messages)


In [4]:
response

AIMessage(content='Bonjour, comment allez-vous ? \n\n\n(Formal)\n\nOr, for a more casual tone:\n\nSalut, ça va ? \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 22, 'total_tokens': 53, 'completion_time': 0.056363636, 'prompt_time': 0.002173656, 'queue_time': 0.24883436300000003, 'total_time': 0.058537292}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-9df347a8-a4f9-492c-8870-2e6a89638c31-0', usage_metadata={'input_tokens': 22, 'output_tokens': 31, 'total_tokens': 53})

The StrOutputParser from langchain_core.output_parsers in the LangChain framework is used to parse and format the output generated by a language model (LLM) or any other process in a chain into a string format.

Function of StrOutputParser:
Output Handling: It processes the raw output from a model or an operation (such as text generation) and returns it as a clean, human-readable string.

Simplifies Response: It simplifies and structures the model's output by ensuring it is in a proper string format, often removing unnecessary metadata or tokens.

In [5]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
parser.invoke(response)

'Bonjour, comment allez-vous ? \n\n\n(Formal)\n\nOr, for a more casual tone:\n\nSalut, ça va ? \n'

In [6]:
### Using LCEL -- chain the components
chain = model|parser
chain.invoke(messages)

'Bonjour, comment allez-vous ?  \n'

In [7]:
### Prompt Template
from langchain_core.prompts import ChatPromptTemplate

generic_template = "Translate the following into {language}"

prompt = ChatPromptTemplate.from_messages(
    [("system", generic_template),
     ("human", "{text}")] )

In [8]:
result = prompt.invoke({"language": "French", "text": "Hello?"})

In [9]:
result.to_messages()

[SystemMessage(content='Translate the following into French', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello?', additional_kwargs={}, response_metadata={})]

In [10]:
## Chaining together with LCEL 
chain = prompt | model | parser  
chain.invoke({"language": "French", "text": "Hello?"})

"Bonjour? \n\n\nLet me know if you have any other phrases you'd like translated!\n"

In [11]:
import langserve
print(langserve.__version__)

0.3.1


In [9]:
!pip uninstall langserve -y
!pip uninstall pydantic -y
!pip uninstall fastapi -y

Found existing installation: langserve 0.0.27
Uninstalling langserve-0.0.27:
  Successfully uninstalled langserve-0.0.27
Found existing installation: pydantic 1.10.8
Uninstalling pydantic-1.10.8:
  Successfully uninstalled pydantic-1.10.8
Found existing installation: fastapi 0.99.1
Uninstalling fastapi-0.99.1:
  Successfully uninstalled fastapi-0.99.1


In [11]:
!pip install --upgrade langserve
!pip install --upgrade pydantic
!pip install --upgrade fastapi


  Using cached langserve-0.3.1-py3-none-any.whl.metadata (40 kB)
  Using cached pydantic-2.11.3-py3-none-any.whl.metadata (65 kB)
Using cached langserve-0.3.1-py3-none-any.whl (1.2 MB)
Using cached pydantic-2.11.3-py3-none-any.whl (443 kB)
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.8
    Uninstalling pydantic-1.10.8:
      Successfully uninstalled pydantic-1.10.8
  Attempting uninstall: langserve
    Found existing installation: langserve 0.0.27
    Uninstalling langserve-0.0.27:
      Successfully uninstalled langserve-0.0.27
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastapi 0.99.1 requires pydantic!=1.8,!=1.8.1,<2.0.0,>=1.7.4, but you have pydantic 2.11.3 which is incompatible.
  Using cached fastapi-0.115.12-py3-none-any.whl.metadata (27 kB)
  Using cached starlette-0.46.1-py3-none-any.whl.metadata (6.2 kB)
  Attemp